In [2]:
import numpy as np
import pandas as pd
from numba import jit
import time

# European call, put option

euro call option

In [18]:
@jit
def binom_tree_european_call_put(S0, k, T, r, sigma, N, call = True, array_out = False):
    dt = T/N
    u = np.exp(sigma*np.sqrt(dt))
    d = 1/u
    R = np.exp(r*dt)
    q = (R-d)/(u-d)
    print('u: {}, d :{}, R : {}, q :{}'.format(u, d, R, q))
    
    price_tree = np.zeros([N+1,N+1])
    for i in range(N+1):
        for j in range(i+1):
            price_tree[j,i] = S0*(d**j)*(u**(i-j))
            
    option = np.zeros([N+1,N+1])
    if call:
        option[:,N] = np.maximum(np.zeros(N+1),price_tree[:,N]-k)
    else:
        option[:,N] = np.maximum(np.zeros(N+1),k-price_tree[:,N])

    for i in np.arange(N-1,-1,-1):
        for j in np.arange(0,i+1):
            option[j,i] = np.exp(-r*dt)*(q*option[j,i+1]+(1-q)*option[j+1,i+1])
            
    if array_out:
        return option[0,0],price_tree,option
    else:
        return option[0,0], option

In [21]:
price, price_tree, option_tree  = binom_tree_european_call_put(60, 57, 10, 0.05, 0.2, 10, True, True)
stock_price_1 = pd.DataFrame(price_tree)
euro_call = pd.DataFrame(option_tree)
print('euro call option price : {}'.format(price))
euro_call

u: 1.2214027581601699, d :0.8187307530779818, R : 1.0512710963760241, q :0.5774931963561243
euro call option price : 28.224792514143704


,0,1,2,3,4,5,6,7,8,9,10
0,28.224793,38.620061,52.148846,69.482715,91.376535,118.705265,152.539362,194.251643,245.606213,308.758771,386.343366
1,0.000000,17.441344,24.815103,34.784715,47.989389,65.111441,86.864803,114.036555,147.631283,189.091921,240.181945
2,0.000000,0.000000,9.479202,14.199686,20.957343,30.410028,43.279642,60.266773,81.956723,108.876833,142.207015
3,0.000000,0.000000,0.000000,4.177420,6.686275,10.580339,16.509757,25.313166,37.933749,55.107051,76.532456
4,0.000000,0.000000,0.000000,0.000000,1.255181,2.175146,3.759780,6.480484,11.134774,19.064088,32.509482
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.150066,0.273181,0.497299,0.905285,1.647986,3.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [22]:
stock_price_1

,0,1,2,3,4,5,6,7,8,9,10
0,60.0,73.284165,89.509482,109.327128,133.532456,163.096910,199.207015,243.311998,297.181945,362.978848,443.343366
1,0.0,49.123845,60.000000,73.284165,89.509482,109.327128,133.532456,163.096910,199.207015,243.311998,297.181945
2,0.0,0.000000,40.219203,49.123845,60.000000,73.284165,89.509482,109.327128,133.532456,163.096910,199.207015
3,0.0,0.000000,0.000000,32.928698,40.219203,49.123845,60.000000,73.284165,89.509482,109.327128,133.532456
4,0.0,0.000000,0.000000,0.000000,26.959738,32.928698,40.219203,49.123845,60.000000,73.284165,89.509482
5,0.0,0.000000,0.000000,0.000000,0.000000,22.072766,26.959738,32.928698,40.219203,49.123845,60.000000
6,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,18.071653,22.072766,26.959738,32.928698,40.219203
7,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,14.795818,18.071653,22.072766,26.959738
8,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,12.113791,14.795818,18.071653
9,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.917933,12.113791


euro put option

In [23]:
#S0 = 100, K = 97, T = 5, r = 0.04, σ = 0.1, N = 10
price, price_tree, option_tree  = binom_tree_european_call_put(100, 97, 5, 0.04, 0.1, 10, False, True)
euro_put = pd.DataFrame(option_tree)
stock_price_2 = pd.DataFrame(price_tree)
print('euro put option price : {}'.format(price))
euro_put

u: 1.073270660257192, d :0.9317314234233945, R : 1.0202013400267558, q :0.6250557695690214
euro put option price : 1.4934544425996434


,0,1,2,3,4,5,6,7,8,9,10
0,1.493454,0.812662,0.370419,0.127264,0.025105,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,2.708844,1.593693,0.795731,0.304426,0.068309,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,4.713820,3.009810,1.657641,0.714450,0.185865,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,7.808483,5.426126,3.319312,1.634127,0.505728,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,12.200732,9.230668,6.307459,3.603285,1.376055,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,17.809371,14.601174,11.155324,7.510357,3.744165,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,24.117176,21.132310,17.832744,14.193482,10.187655
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,30.392530,27.771466,24.860421,21.636168
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,36.399504,34.120641,31.574891
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,42.159655,40.202929


In [24]:
stock_price_2

,0,1,2,3,4,5,6,7,8,9,10
0,100.0,107.327066,115.190991,123.631111,132.689644,142.411902,152.846516,164.045681,176.065417,188.965846,202.811498
1,0.0,93.173142,100.000000,107.327066,115.190991,123.631111,132.689644,142.411902,152.846516,164.045681,176.065417
2,0.0,0.000000,86.812345,93.173142,100.000000,107.327066,115.190991,123.631111,132.689644,142.411902,152.846516
3,0.0,0.000000,0.000000,80.885789,86.812345,93.173142,100.000000,107.327066,115.190991,123.631111,132.689644
4,0.0,0.000000,0.000000,0.000000,75.363832,80.885789,86.812345,93.173142,100.000000,107.327066,115.190991
5,0.0,0.000000,0.000000,0.000000,0.000000,70.218850,75.363832,80.885789,86.812345,93.173142,100.000000
6,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,65.425109,70.218850,75.363832,80.885789,86.812345
7,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,60.958630,65.425109,70.218850,75.363832
8,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,56.797071,60.958630,65.425109
9,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,52.919616,56.797071


# American call, put option

In [25]:
@jit
def binom_tree_american_call_put(S0, k, T, r, sigma, N, call = True, array_out = False):
    dt = T/N
    u = np.exp(sigma*np.sqrt(dt))
    d = 1/u
    R = np.exp(r*dt)
    
    q = (R-d)/(u-d)
    
    price_tree = np.zeros([N+1,N+1])
    for i in range(N+1):
        for j in range(i+1):
            price_tree[j,i] = S0*(d**j)*(u**(i-j))
            
    option = np.zeros([N+1,N+1])
    if call:
        option[:,N] = np.maximum(np.zeros(N+1),price_tree[:,N]-k)
    else:
        option[:,N] = np.maximum(np.zeros(N+1),k-price_tree[:,N])

    for i in np.arange(N-1,-1,-1):
        for j in np.arange(0,i+1):
            if call :
                option[j,i] = max(np.exp(-r*dt)*(q*option[j,i+1]+(1-q)*option[j+1,i+1]),price_tree[j,i]-k )
            else:
                option[j,i] = max(np.exp(-r*dt)*(q*option[j,i+1]+(1-q)*option[j+1,i+1]), k - price_tree[j,i])
                
            
    if array_out:
        return option[0,0], price_tree, option
    else:
        return option[0,0], option

american call option

In [27]:
price, price_tree, option_tree  = binom_tree_american_call_put(60, 57, 10, 0.05, 0.2, 10, True, True)
ameri_call = pd.DataFrame(option_tree)
stock_price_3 = pd.DataFrame(price_tree)
print('american call option price : {}'.format(price))
ameri_call

american call option price : 28.224792514143704


,0,1,2,3,4,5,6,7,8,9,10
0,28.224793,38.620061,52.148846,69.482715,91.376535,118.705265,152.539362,194.251643,245.606213,308.758771,386.343366
1,0.000000,17.441344,24.815103,34.784715,47.989389,65.111441,86.864803,114.036555,147.631283,189.091921,240.181945
2,0.000000,0.000000,9.479202,14.199686,20.957343,30.410028,43.279642,60.266773,81.956723,108.876833,142.207015
3,0.000000,0.000000,0.000000,4.177420,6.686275,10.580339,16.509757,25.313166,37.933749,55.107051,76.532456
4,0.000000,0.000000,0.000000,0.000000,1.255181,2.175146,3.759780,6.480484,11.134774,19.064088,32.509482
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.150066,0.273181,0.497299,0.905285,1.647986,3.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [28]:
stock_price_3

,0,1,2,3,4,5,6,7,8,9,10
0,60.0,73.284165,89.509482,109.327128,133.532456,163.096910,199.207015,243.311998,297.181945,362.978848,443.343366
1,0.0,49.123845,60.000000,73.284165,89.509482,109.327128,133.532456,163.096910,199.207015,243.311998,297.181945
2,0.0,0.000000,40.219203,49.123845,60.000000,73.284165,89.509482,109.327128,133.532456,163.096910,199.207015
3,0.0,0.000000,0.000000,32.928698,40.219203,49.123845,60.000000,73.284165,89.509482,109.327128,133.532456
4,0.0,0.000000,0.000000,0.000000,26.959738,32.928698,40.219203,49.123845,60.000000,73.284165,89.509482
5,0.0,0.000000,0.000000,0.000000,0.000000,22.072766,26.959738,32.928698,40.219203,49.123845,60.000000
6,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,18.071653,22.072766,26.959738,32.928698,40.219203
7,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,14.795818,18.071653,22.072766,26.959738
8,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,12.113791,14.795818,18.071653
9,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.917933,12.113791


american put option

In [29]:
price, price_tree, option_tree  = binom_tree_american_call_put(100, 97, 5, 0.04, 0.1, 10, False, True)
ameri_put = pd.DataFrame(option_tree)
stock_price_4 = pd.DataFrame(price_tree)
print('american put option price : {}'.format(price))
ameri_put

american put option price : 2.7163923308348097


,0,1,2,3,4,5,6,7,8,9,10
0,2.716392,1.243363,0.488118,0.146914,0.025659,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,5.318380,2.569396,1.083225,0.356968,0.069818,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,10.187655,5.185371,2.352302,0.854899,0.189970,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,16.114211,10.187655,4.975306,2.009439,0.516897,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,21.636168,16.114211,10.187655,4.605865,1.406446,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,26.781150,21.636168,16.114211,10.187655,3.826858,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,31.574891,26.781150,21.636168,16.114211,10.187655
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,36.041370,31.574891,26.781150,21.636168
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,40.202929,36.041370,31.574891
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,44.080384,40.202929


In [30]:
stock_price_4

,0,1,2,3,4,5,6,7,8,9,10
0,100.0,107.327066,115.190991,123.631111,132.689644,142.411902,152.846516,164.045681,176.065417,188.965846,202.811498
1,0.0,93.173142,100.000000,107.327066,115.190991,123.631111,132.689644,142.411902,152.846516,164.045681,176.065417
2,0.0,0.000000,86.812345,93.173142,100.000000,107.327066,115.190991,123.631111,132.689644,142.411902,152.846516
3,0.0,0.000000,0.000000,80.885789,86.812345,93.173142,100.000000,107.327066,115.190991,123.631111,132.689644
4,0.0,0.000000,0.000000,0.000000,75.363832,80.885789,86.812345,93.173142,100.000000,107.327066,115.190991
5,0.0,0.000000,0.000000,0.000000,0.000000,70.218850,75.363832,80.885789,86.812345,93.173142,100.000000
6,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,65.425109,70.218850,75.363832,80.885789,86.812345
7,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,60.958630,65.425109,70.218850,75.363832
8,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,56.797071,60.958630,65.425109
9,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,52.919616,56.797071
